In [1]:
import numpy as np
from scipy.optimize import minimize

# Optimization Design homework 4
## Problem 1
let x1 be the radius and x2 be the length . 
maximize : 2 * x_1 * x_2 + ( x_1 ** 2 * pi ) / 2   
s.t : 2x_1 + 2x_2 + x_1*pi = 10  
      x_1 > 0  
      x_2 > 0   

### Define objective function
Hence maximize objective function is equal to minimize negative of objective function

In [2]:
def objective1(x):
    '''
    objective function of problem 1
    Args:
    ---
        x (2d numpy array) : input vector
    Return:
    ---
        value : function value
    '''
    return ( 2 * x[0] * x[1] + ( x[0] ** 2 * np.pi ) / 2 ) * -1

### Define bounds of x_1 and x_2

In [3]:
from scipy.optimize import Bounds
bounds_1 = Bounds([1, 1], [np.inf, np.inf])

### Define Linear Constraints

In [4]:
from scipy.optimize import LinearConstraint
linear_constraints_1 = LinearConstraint([2+np.pi, 2], 10, 10)

### Solve optimization problem

In [5]:
x0 = np.array([1, 1])
res = minimize(objective1, x0, method = 'trust-constr', constraints=[linear_constraints_1], options={'disp':True}, bounds=bounds_1)

`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 21, CG iterations: 6, optimality: 8.15e-09, constraint violation: 0.00e+00, execution time: 0.024 s.


### Print optimum point

In [6]:
print(res.x)

[1.40023028 1.40029313]


### Check funtion value of optimum point
whether it is minimum or not

In [7]:
objective1(res.x)

-7.00123941778847

In [8]:
objective1(res.x - np.array([0.1, 0.1]))

-6.036947381056418

### Check constraint

In [9]:
temp = np.array([2+np.pi, 2])
temp @ res.x

10.0

## Problem 2
### Define objective function

In [55]:
def objective_2(x):
    '''
    objective function of problem2
    Args:
    ---
        x : input vector
    Return:
    ---
        value : function value at vector x
    '''
    A = np.array([11.5, 92.5, 44.3, 98.1, 20.1, 6.1, 45.5, 31.0, 44.3])
    value = np.sum(np.divide(x, A)**2)
    return -1*value 


### Define bounds of F_i

In [27]:
bounds_F = Bounds([0,0,0,0,0,0,0,0,0], [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf])

### Define Linear Constraints

In [47]:
linear_constraints_D = LinearConstraint([[0.0298, -0.044, -0.044, 0, 0, 0, 0, 0, 0], 
                                         [0, 0, -0.0138, 0.0329, 0.0329, -0.025, -0.025, 0, 0],
                                         [0, 0, 0, 0, 0.0279, 0, -0.0619, 0.0317, -0.0368]
                                        ], [4, 33, 31], [4, 33, 31])

### Solve optimization problem

In [56]:
np.random.seed(1024)
F1 = np.random.rand(9)
prob2_res = minimize(objective_2, F1, method='trust-constr', constraints=[linear_constraints_D], options={'disp':True}, bounds=bounds_F)

`xtol` termination condition is satisfied.
Number of iterations: 651, function evaluations: 2990, CG iterations: 1022, optimality: 7.52e+43, constraint violation: 7.90e+44, execution time:  1.3 s.


In [57]:
print(prob2_res.x)

[ 6.07445733e+45  1.79204078e+45  2.32202350e+45 -7.90291619e+44
  1.78521470e+45  2.38137526e+43  3.74804650e+42  4.66912089e+30
  1.34715994e+45]


## Problem 3
Solve the first problem numerically on the following [website](http://people.brunel.ac.uk/~mastjjb/jeb/or/morelp.html)  
- Use Linear programming method
- Use Non linear programming method

### Problem 3-1

In [10]:
from scipy.optimize import linprog
c = [-1,-1]
A = [[50,24], [30,33]]
b = [40*60,35*60]
x0_bounds = (45, None)
x1_bounds = (5, None)

In [11]:
res3_1 = linprog(c, A_ub = A, b_ub = b, bounds=[x0_bounds, x1_bounds], method='revised simplex')
print(res3_1)

con: array([], dtype=float64)
     fun: -51.25
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([  0.  , 543.75])
  status: 0
 success: True
       x: array([45.  ,  6.25])


In [12]:
print(f'Actual Optimum Value : {res3_1.fun * -1 - 50}')
print(f'Optimal point (x_0, x_1) : ({res3_1.x[0]},{res3_1.x[1]})')

Actual Optimum Value : 1.25
Optimal point (x_0, x_1) : (45.0,6.25)


### Problem 3-2
Here i use exterior penalty function method

#### Define combined objective

In [96]:
def main_obj(x):
    '''
    combine main objective and penalty function
    Args:
    ---
        x : input vector 
    Return:
    ---
        value : objective function value
    '''
    # ==== original objective and penalty function ====
    def objective3(x):
        '''
        objective of problem3-2
        Args:
        ---
            x : input vector
        Return:
        ---
            value : function value of input vector
        '''
        value = x[0] + x[1] - 50
        return value

    def g1(x):
        '''
        First penalty function
        '''
        return 50 * x[0] + 24 * x[1] - 2400
    
    def g2(x):
        '''
        Second penalty function
        '''
        return 30 * x[0] + 33 * x[1] - 2100

    def g3(x):
        '''
        Third penalty function
        '''
        return 45 - x[0]
    
    def g4(x):
        '''
        Fourth penalty function
        '''
        return 5 - x[1]
    # =================================================
    r = x[-1]
    #TODO : convert to exterior method
    return value

#### Implement algorithm